In [36]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.optimize import dual_annealing

In [32]:
data = pd.read_excel(r'Тестовая_таблица_графы.xlsx', sheet_name='Граф')
alpha_value = pd.read_excel(r'Тестовая_таблица_графы.xlsx', sheet_name='Объем')

In [33]:
data['Вес_связи'] = np.nan

In [34]:
data = data.merge(alpha_value, left_on='Родитель', right_on='Альфа')
data = data.rename(columns={'Объем': 'Факт_Родитель'})
data = data.drop(['Альфа'], axis=1)

data = data.merge(alpha_value, left_on='Потомок', right_on='Альфа')
data = data.rename(columns={'Объем': 'Факт_Потомок'})
data = data.drop(['Альфа'], axis=1)

In [126]:
data

,Колонка_1,Родитель,Потомок,Вес_связи,Факт_Родитель,Факт_Потомок
0,Тест,Икс_1,Икс_2,0.477295,10000,5000
1,Тест,Икс_8,Икс_2,0.227051,1000,5000
2,Тест,Икс_1,Икс_3,0.500000,10000,5000
3,Тест,Икс_2,Икс_4,0.400000,5000,2000
4,Тест,Икс_2,Икс_5,0.600000,5000,3000
5,Тест,Икс_4,Икс_6,0.250000,2000,500
6,Тест,Икс_4,Икс_7,0.750000,2000,1500
7,Тест,Икс_5,Икс_8,0.333341,3000,1000
8,Тест,Икс_5,Икс_9,0.666667,3000,2000
9,Тест,Икс_8,Икс_10,0.200000,1000,200


In [123]:
def func_minimize(weights):
    data['Вес_связи'] = weights
    '''
    for parent in data['Родитель'].unique():
        sum_weight = data[data['Родитель'] == parent]['Вес_связи'].sum()
        if sum_weight > 1:
            return 10000000000
        elif sum_weight < 1:
            return 10000000000
    '''
    
    MSE_value = 0
    for descendant in data['Потомок'].unique():
        weights_descendant = data[data['Потомок'] == descendant]['Вес_связи'].values
        fact_parent = data[data['Потомок'] == descendant]['Факт_Родитель'].values
        fact_descendant = data[data['Потомок'] == descendant]['Факт_Потомок'].values
        
        var_1 = fact_parent * weights_descendant
        var = fact_descendant - var_1.sum()
        
        MSE_value += var[0] ** 2   
    return MSE_value

In [124]:
lw = [0] * 11
up = [1] * 11
ret = dual_annealing(func_minimize, bounds=list(zip(lw, up)))

In [125]:
ret.fun

1.0042116821710939e-08